In [30]:
import pandas as pd
import numpy as np
import re
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder,Imputer
from sklearn.model_selection import train_test_split

In [11]:
#train set
df_train = pd.read_csv('data/busara/train.csv')
# test set
df_test = pd.read_csv('data/busara/test.csv')
#show the shape of the train dataframe
df_train.shape

(1143, 75)

In [12]:
def missing_values_table(df):
    # Total missing values
    mis_val = df.isnull().sum()
 
    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)
 
    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
 
    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
 
    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
    mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
 
    # Print some summary information
    print ("The dataset has " + str(df.shape[1]) + " columns." 
    "There are " + str(mis_val_table_ren_columns.shape[0]) +
    " columns that have missing values.")
 
    # Return the dataframe with missing information
    return mis_val_table_ren_columns

In [14]:
# Get the columns with > 50% missing
missing_df = missing_values_table(df_train);
missing_columns = list(missing_df[missing_df['% of Total Values'] > 50].index)
print('','%d columns will be deleted.' % len(missing_columns))
# Drop the columns with 50% missing data
df_train = df_train.drop(list(missing_columns),axis=1)
df_train.shape

The dataset has 71 columns.There are 19 columns that have missing values.
 0 columns will be deleted.


(1143, 71)

In [17]:
# Create a label (category) encoder object
encoder = LabelEncoder()
# fitting the encoder to the "survey_date" column
encoder.fit(df_train['survey_date'])
# Apply the fitted encoder to the "survey_date" to transform categories into integers
encoded_train = encoder.transform(df_train['survey_date'])
# encoded_test = encoder.transform(df_test['survey_date'])
#assign the tranformed column back to the dataframe
df_train['survey_date'] = encoded_train

In [24]:
# split data into train and test sets
X = df_train.drop(["depressed"], axis=1)
# fill missing values with mean column values
imputer = Imputer()
transformed_X = imputer.fit_transform(X)
y = df_train.depressed
seed = 5
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(transformed_X, y, test_size=test_size, random_state=seed)

d:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [40]:
# create model
model = Sequential()
model.add(Dense(36, input_dim=70, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(36, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [41]:
# Fit the model
model.fit(X_train, y_train, epochs=100, batch_size=10)

Epoch 1/100
765/765 [==============================] - 0s - loss: 4.6527 - acc: 0.6837     
Epoch 2/100
765/765 [==============================] - 0s - loss: 2.9802 - acc: 0.8105     
Epoch 3/100
765/765 [==============================] - 0s - loss: 3.0310 - acc: 0.8065     
Epoch 4/100
765/765 [==============================] - 0s - loss: 2.9770 - acc: 0.8144     
Epoch 5/100
765/765 [==============================] - 0s - loss: 2.9922 - acc: 0.8144     
Epoch 6/100
765/765 [==============================] - 0s - loss: 3.0113 - acc: 0.8131     
Epoch 7/100
765/765 [==============================] - 0s - loss: 2.9720 - acc: 0.8144     
Epoch 8/100
765/765 [==============================] - 0s - loss: 3.0131 - acc: 0.8118     
Epoch 9/100
765/765 [==============================] - 0s - loss: 2.9841 - acc: 0.8131     
Epoch 10/100
765/765 [==============================] - 0s - loss: 2.9919 - acc: 0.8144     
Epoch 11/100
765/765 [==============================] - 0s - loss: 2.9956 - acc

In [43]:
scores = model.evaluate(X_test, y_test)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

 32/378 [=>............................] - ETA: 2sacc: 86.24%
